In [40]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [41]:
from sub import subMNIST       # testing the subclass of MNIST dataset

# Split Data

In [3]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                             ])

# Train Model

In [17]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [44]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

In [45]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [46]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = Net()

In [50]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [51]:
# CPU only training
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [52]:
for epoch in range(1, 100):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 2.281664
Train Epoch: 1 [640/3000 (21%)]	Loss: 2.262927
Train Epoch: 1 [1280/3000 (43%)]	Loss: 2.220351
Train Epoch: 1 [1920/3000 (64%)]	Loss: 2.220092
Train Epoch: 1 [2560/3000 (85%)]	Loss: 2.243490

Test set: Average loss: 2.0880, Accuracy: 4712/10000 (47%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 2.174645
Train Epoch: 2 [640/3000 (21%)]	Loss: 2.006025
Train Epoch: 2 [1280/3000 (43%)]	Loss: 2.131283
Train Epoch: 2 [1920/3000 (64%)]	Loss: 1.956879
Train Epoch: 2 [2560/3000 (85%)]	Loss: 1.943989

Test set: Average loss: 1.6537, Accuracy: 6915/10000 (69%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 1.687298
Train Epoch: 3 [640/3000 (21%)]	Loss: 1.557825
Train Epoch: 3 [1280/3000 (43%)]	Loss: 1.821832
Train Epoch: 3 [1920/3000 (64%)]	Loss: 1.555201
Train Epoch: 3 [2560/3000 (85%)]	Loss: 1.506146

Test set: Average loss: 1.0487, Accuracy: 7769/10000 (78%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 1.361330
Train Epoch: 4 [640/3000 (21%)]	Loss: 1.327900
Train Epoch: 4 [1280/

# Create Sample Submission

In [27]:
testset = datasets.MNIST('../data', train=False, transform=transform)

In [28]:
pickle.dump(testset, open("test.p", "wb" ))

In [29]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [30]:
test(1, test_loader)


Test set: Average loss: 0.2851, Accuracy: 9203/10000 (92%)



In [31]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [32]:
label_predict

array([ 7.,  2.,  1., ...,  4.,  5.,  6.])

In [33]:
label_true = test_loader.dataset.test_labels.numpy()

In [34]:
diff_array = label_true - label_predict

In [35]:
len(np.where(diff_array != 0)[0])

797

In [36]:
import pandas as pd
true_label = pd.DataFrame(label_true, columns=['label'])
true_label.reset_index(inplace=True)
true_label.rename(columns={'index': 'ID'}, inplace=True)

In [37]:
true_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [38]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [39]:
predict_label.head()

,ID,label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4


In [53]:
predict_label.to_csv('sample_submission.csv', index=False)
true_label.to_csv('true_label.csv', index=False)